In [1]:
import pandas as pd
import ast
import re
from unidecode import unidecode
from rapidfuzz import process, fuzz

In [2]:
import pandas as pd
import ast
import re
from unidecode import unidecode
from rapidfuzz import process, fuzz

# ===================================================
# 1️⃣ HÀM ĐẾM SỐ DÒNG CÓ URL NHAC.VN
# ===================================================
def count_nhacvn_urls(df):
    count_with_nhacvn = 0
    total = len(df)
    for urls in df['urls']:
        if pd.isna(urls):
            continue
        try:
            parsed = ast.literal_eval(urls)
            if isinstance(parsed, str):
                parsed = [parsed]
        except Exception:
            parsed = [urls]
        if any(isinstance(u, str) and "nhac.vn/bai-hat/" in u for u in parsed):
            count_with_nhacvn += 1
    return count_with_nhacvn, total

# ===================================================
# 2️⃣ XOÁ CÁC URL NHAC.VN + THÊM CỘT MARK
# ===================================================
def remove_nhacvn_urls_add_mark(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    before, total = count_nhacvn_urls(df)
    print(f"🎵 Trước khi xóa: {before}/{total} dòng chứa URL nhac.vn ({before/total*100:.2f}%)")

    marks = []

    def clean_urls(urls):
        if pd.isna(urls) or not isinstance(urls, str):
            marks.append(0)
            return '[]'
        try:
            parsed = ast.literal_eval(urls)
            if isinstance(parsed, str):
                parsed = [parsed]
        except Exception:
            parsed = [urls]
        has_nhacvn = any(isinstance(u, str) and "nhac.vn/bai-hat/" in u for u in parsed)
        filtered = [u for u in parsed if isinstance(u, str) and "nhac.vn/bai-hat/" not in u]
        marks.append(1 if has_nhacvn else 0)
        return str(filtered) if filtered else '[]'

    df['urls'] = df['urls'].apply(clean_urls)

    # Thêm cột mark sau cột note (nếu có)
    insert_pos = df.columns.get_loc('note') + 1 if 'note' in df.columns else len(df.columns)
    df.insert(insert_pos, 'mark', marks)

    after, _ = count_nhacvn_urls(df)
    print(f"🧹 Sau khi xóa: {after} dòng còn chứa nhac.vn")
    print(f"📍 Tổng số dòng mark=1 (đã xóa nhacvn): {sum(marks)}")

    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"💾 Đã lưu file cleaned + mark tại: {output_csv}")

    return df

# ===================================================
# 3️⃣ CHUẨN HÓA TEXT & RÚT GỌN LYRICS
# ===================================================
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    text = unidecode(text.lower())
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def make_signature(title, lyrics, max_words=30):
    """Ghép title và 20-30 ký tự đầu của lyrics để so khớp."""
    lyrics_part = ' '.join(lyrics.split()[:max_words]) if isinstance(lyrics, str) else ''
    return normalize_text(f"{title} {lyrics_part}")

# ===================================================
# 4️⃣ FILL LẠI URL DỰA TRÊN MARK==1
# ===================================================
def refill_urls_marked(final_csv, reference_csv, preview_csv, output_csv, threshold=90):
    df_final = pd.read_csv(final_csv)
    df_ref = pd.read_csv(reference_csv)

    df_ref['signature'] = df_ref.apply(lambda r: make_signature(r['title'], r.get('lyrics', '')), axis=1)
    ref_signatures = df_ref['signature'].tolist()
    ref_urls = df_ref['urls'].tolist()

    preview_rows = []
    matched_count = 0

    # Chỉ xét các hàng bị xóa nhac.vn
    target_rows = df_final[df_final['mark'] == 1]

    before_fill, total = count_nhacvn_urls(df_final)
    print(f"🔍 Trước khi fill lại: {before_fill}/{total} dòng có URL nhac.vn")

    for i, row in target_rows.iterrows():
        sig = make_signature(row['title'], row.get('lyrics', ''))
        if not sig:
            continue

        best_match = process.extractOne(sig, ref_signatures, scorer=fuzz.token_set_ratio)
        if not best_match:
            continue

        ref_sig, score, idx = best_match
        if score >= threshold:
            matched_url = df_ref.iloc[idx]['urls']
            preview_rows.append({
                'index': i,
                'title': row['title'],
                'url_new': matched_url,
                'similarity': score
            })
            matched_count += 1
            df_final.at[i, 'urls'] = str([matched_url])

    df_preview = pd.DataFrame(preview_rows)
    df_preview.to_csv(preview_csv, index=False, encoding='utf-8-sig')

    after_fill, _ = count_nhacvn_urls(df_final)
    print(f"✅ Đã gán thêm {matched_count} URL nhac.vn mới (threshold ≥ {threshold})")
    print(f"📈 Sau khi fill lại: {after_fill}/{total} dòng có URL nhac.vn")

    df_final.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"💾 Đã lưu final_dataset mới tại: {output_csv}")
    print(f"👁️‍🗨️ Preview lưu tại: {preview_csv}")

    return df_preview, df_final

# ===================================================
# 🚀 CHẠY QUY TRÌNH
# ===================================================
# Bước 1: Xóa + đánh dấu mark
cleaned_df = remove_nhacvn_urls_add_mark("final_dataset_cleaned_v2.csv", "final_dataset_no_nhacvn.csv")

# Bước 2: Fill lại dựa trên title + 20-30 từ đầu của lyrics
#preview, final = refill_urls_marked(
 #   final_csv="final_dataset_no_nhacvn.csv",
  #  reference_csv="outputNhacvn2.csv",
   # preview_csv="preview_refill_marked.csv",
    ##hreshold=85
#)


🎵 Trước khi xóa: 20671/62289 dòng chứa URL nhac.vn (33.19%)
🧹 Sau khi xóa: 0 dòng còn chứa nhac.vn
📍 Tổng số dòng mark=1 (đã xóa nhacvn): 20671
💾 Đã lưu file cleaned + mark tại: final_dataset_no_nhacvn.csv


In [4]:
import pandas as pd
import re
from unidecode import unidecode
from rapidfuzz import process, fuzz
import ast
from math import ceil

# -------------------------
# Helpers
# -------------------------
def normalize_text_simple(s):
    """Lower, remove diacritics, remove non-alnum (keeps spaces)."""
    if not isinstance(s, str):
        return ''
    s = unidecode(s).lower()
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def signature_from_parts(title, creator, lyrics_snip, max_chars=30):
    """Build a normalized signature from title + creator + short lyrics snippet."""
    title_n = normalize_text_simple(title)
    creator_n = normalize_text_simple(creator)
    lyrics_n = normalize_text_simple((lyrics_snip or '')[:max_chars])
    # order: title, creator, lyrics snippet
    return (title_n + ' ' + creator_n + ' ' + lyrics_n).strip()

def safe_eval_list(x):
    """Try to interpret x as Python list; fallback to [x] or []."""
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    try:
        parsed = ast.literal_eval(x)
        if isinstance(parsed, (list, tuple)):
            return list(parsed)
        if isinstance(parsed, str):
            return [parsed]
        return [str(parsed)]
    except Exception:
        # fallback: try split by comma if it looks like multiple, else single
        if isinstance(x, str) and ',' in x:
            parts = [p.strip() for p in x.split(',') if p.strip()]
            return parts
        return [x] if x else []

# -------------------------
# Main function
# -------------------------
def refill_using_title_composer_lyrics(
    final_csv,
    reference_csv,
    preview_csv="preview_replace_title_comp_lyrics.csv",
    output_filled_csv="final_dataset_filled.csv",
    batch_size=4000,
    similarity_threshold=85,
    lyrics_chars=30,
    do_replace=False,
    save_partial_every=1
):
    """
    - final_csv: final_dataset_no_nhacvn.csv (must have 'mark' column)
    - reference_csv: outputNhacvn2.csv (must have 'urls','title','composer','lyrics' columns)
    - preview_csv: output preview mapping index -> new url (saved progressively per batch)
    - do_replace: if True, apply replacements into a copy and save output_filled_csv
    - similarity_threshold: minimal score to accept a match (0-100)
    - lyrics_chars: number of characters taken from lyrics for signature
    """
    print("🔄 Loading data...")
    df_final = pd.read_csv(final_csv)
    df_ref = pd.read_csv(reference_csv)

    # Ensure required columns exist
    for c in ['urls', 'title']:
        if c not in df_ref.columns:
            raise ValueError(f"Reference file missing required column: {c}")

    # Normalize and prepare reference dataframe (reset index)
    df_ref = df_ref[['urls', 'title'] + ([c for c in ['composer','lyrics'] if c in df_ref.columns])].copy()
    df_ref = df_ref.reset_index(drop=True)
    df_ref['title_n'] = df_ref['title'].astype(str).apply(normalize_text_simple)
    df_ref['composer_n'] = df_ref['composer'].astype(str).apply(normalize_text_simple) if 'composer' in df_ref.columns else ''
    df_ref['lyrics_n'] = df_ref['lyrics'].astype(str).apply(normalize_text_simple) if 'lyrics' in df_ref.columns else ''
    # Precompute small lyrics snippets used in signature
    df_ref['lyrics_snip'] = df_ref['lyrics_n'].str[:lyrics_chars]

    # Build reference signatures (full) used as fallback pool
    df_ref['full_signature'] = df_ref.apply(
        lambda r: signature_from_parts(r['title'], r.get('composer', ''), r.get('lyrics_snip', ''), max_chars=lyrics_chars),
        axis=1
    )

    # Indexing maps for quick prefilter:
    # by first word of title -> list of indices
    df_ref['first_word'] = df_ref['title_n'].str.split().str[0].fillna('')
    by_first_word = df_ref.groupby('first_word').apply(lambda g: g.index.tolist()).to_dict()
    # by composer exact token -> indices
    df_ref['composer_token'] = df_ref['composer_n'].str.split().str[0].fillna('')
    by_composer = df_ref.groupby('composer_token').apply(lambda g: g.index.tolist()).to_dict()

    # Filter final rows to only those marked (mark==1)
    if 'mark' not in df_final.columns:
        raise ValueError("final dataset must have 'mark' column (1 = needs fill).")
    df_target = df_final[df_final['mark'] == 1].copy()
    print(f"🎯 Rows to process (mark==1): {len(df_target)}")

    # We'll accumulate preview mappings here
    preview_records = []

    # For replacement, operate on a copy if requested
    if do_replace:
        df_out = df_final.copy()
    else:
        df_out = None

    total = len(df_target)
    batches = ceil(total / batch_size) if batch_size > 0 else 1

    processed = 0
    for b in range(batches):
        start = b * batch_size
        end = min(start + batch_size, total)
        batch = df_target.iloc[start:end]
        print(f"\n⚙️ Processing batch {b+1}/{batches} - rows {start}:{end} ...")

        # For each row in batch, create query signature and build candidate pool
        for idx, row in batch.iterrows():
            title = str(row.get('title','')).strip()
            if not title:
                continue
            # choose creator: composer first, else lyricist if present in final dataset
            creator = ''
            if 'composers' in row and pd.notna(row.get('composers','')) and str(row.get('composers','')).strip():
                # composers in final dataset might be string/list; make simple string
                comp_val = row.get('composers')
                try:
                    comp_parsed = ast.literal_eval(comp_val) if isinstance(comp_val, str) and comp_val.strip().startswith('[') else comp_val
                except Exception:
                    comp_parsed = comp_val
                if isinstance(comp_parsed, list):
                    creator = ' '.join(map(str, comp_parsed))
                else:
                    creator = str(comp_parsed)
            elif 'lyricists' in row and pd.notna(row.get('lyricists','')) and str(row.get('lyricists','')).strip():
                lyr_val = row.get('lyricists')
                try:
                    lyr_parsed = ast.literal_eval(lyr_val) if isinstance(lyr_val, str) and lyr_val.strip().startswith('[') else lyr_val
                except Exception:
                    lyr_parsed = lyr_val
                if isinstance(lyr_parsed, list):
                    creator = ' '.join(map(str, lyr_parsed))
                else:
                    creator = str(lyr_parsed)
            creator_n = normalize_text_simple(creator)

            # lyrics snippet: prefer final_dataset lyrics if exists (short), else empty
            lyrics_val = row.get('lyrics','') if 'lyrics' in row else ''
            lyrics_snip = normalize_text_simple(str(lyrics_val))[:lyrics_chars]

            # Build query signature
            query_sig = signature_from_parts(title, creator, lyrics_snip, max_chars=lyrics_chars)

            # Build candidate indices by priority:
            candidate_indices = []

            # 1) If creator token exists and matches some ref composers -> use those first
            comp_token = creator_n.split()[0] if creator_n else ''
            if comp_token and comp_token in by_composer:
                candidate_indices.extend(by_composer[comp_token])

            # 2) Prefilter by first_word of title
            first_word = normalize_text_simple(title).split()[0] if title.split() else ''
            if first_word and first_word in by_first_word:
                # extend but avoid duplicates
                for ci in by_first_word[first_word]:
                    if ci not in candidate_indices:
                        candidate_indices.append(ci)

            # 3) fallback: if still empty or to increase recall, add entire reference
            if not candidate_indices:
                candidate_indices = df_ref.index.tolist()

            # Build candidate signatures list (ordered) and a map to global indices
            candidate_sigs = df_ref.loc[candidate_indices, 'full_signature'].tolist()
            candidate_idx_map = list(candidate_indices)  # candidate_idx_map[pos] -> global idx

            # If candidate_sigs empty, skip
            if not candidate_sigs:
                preview_records.append({
                    'index': idx,
                    'title': title,
                    'urls_old': row.get('urls',''),
                    'url_refer': '',
                    'similarity': 0
                })
                continue

            # Do the fuzzy search on candidate_sigs
            best = process.extractOne(query_sig, candidate_sigs, scorer=fuzz.token_set_ratio)
            matched_url = ''
            score = 0
            if best:
                # best may be (match, score) or (match, score, pos)
                # Normalize unpacking:
                if len(best) == 2:
                    matched_sig, score = best[0], best[1]
                    pos = candidate_sigs.index(matched_sig) if matched_sig in candidate_sigs else None
                else:
                    matched_sig, score, pos = best[0], best[1], best[2]
                # if pos provided, map to global index
                if pos is not None and 0 <= int(pos) < len(candidate_idx_map):
                    global_idx = candidate_idx_map[int(pos)]
                    matched_url = df_ref.at[global_idx, 'urls']
                else:
                    # fallback: try to find matched_sig position
                    try:
                        pos2 = candidate_sigs.index(matched_sig)
                        global_idx = candidate_idx_map[pos2]
                        matched_url = df_ref.at[global_idx, 'urls']
                    except Exception:
                        matched_url = ''
            # Append preview record
            preview_records.append({
                'index': idx,
                'title': title,
                'urls_old': row.get('urls',''),
                'url_refer': matched_url,
                'similarity': int(score)
            })

            # If do_replace and score >= threshold -> perform replacement in df_out
            if do_replace and matched_url and score >= similarity_threshold:
                # replace in df_out: replace any nhac.vn url by the matched_url, keep others
                old_urls = safe_eval_list(df_out.at[idx, 'urls'])
                # remove all nhac.vn links
                filtered = [u for u in old_urls if not (isinstance(u, str) and "nhac.vn/bai-hat/" in u)]
                filtered.insert(0, matched_url)  # insert new ref at front
                df_out.at[idx, 'urls'] = str(filtered)

            processed += 1
            if processed % 500 == 0:
                print(f"  • processed {processed}/{total} rows...")

        # After finishing batch, save partial preview
        # convert to dataframe and save (overwrite) so partial results persist
        df_preview_partial = pd.DataFrame(preview_records)
        df_preview_partial.to_csv(preview_csv, index=False, encoding='utf-8-sig')
        print(f"💾 Saved partial preview ({len(preview_records)} rows) to {preview_csv}")

    # Final summary
    df_preview = pd.DataFrame(preview_records)
    matched_count = df_preview['url_refer'].astype(bool).sum()
    high_conf = df_preview[df_preview['similarity'] >= similarity_threshold].shape[0]

    print("\n✅ Done.")
    print(f"🔎 Total preview rows: {len(df_preview)}")
    print(f"🔗 Found candidate url_refer (any score): {matched_count}")
    print(f"✅ High-confidence matches (>= {similarity_threshold}): {high_conf}")

    # Save final preview
    df_preview.to_csv(preview_csv, index=False, encoding='utf-8-sig')
    print(f"📁 Final preview written to: {preview_csv}")

    # If do_replace, save output file
    if do_replace:
        df_out.to_csv(output_filled_csv, index=False, encoding='utf-8-sig')
        print(f"💾 Replacements applied and saved to: {output_filled_csv}")

    return df_preview

In [5]:
preview = refill_using_title_composer_lyrics(
    final_csv="final_dataset_no_nhacvn.csv",
    reference_csv="outputNhacvn2.csv",
    preview_csv="preview_replace_title_comp_lyrics.csv",
    output_filled_csv="final_dataset_filled.csv",
    batch_size=4000,
    similarity_threshold=90,
    lyrics_chars=50,
    do_replace=False
)

🔄 Loading data...


/var/folders/dm/b6833525473fb0p2gk_x1dmm0000gp/T/ipykernel_41590/3147350715.py:97: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_first_word = df_ref.groupby('first_word').apply(lambda g: g.index.tolist()).to_dict()
/var/folders/dm/b6833525473fb0p2gk_x1dmm0000gp/T/ipykernel_41590/3147350715.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_composer = df_ref.groupby('composer_token').apply(lambda g:

🎯 Rows to process (mark==1): 20671

⚙️ Processing batch 1/6 - rows 0:4000 ...
  • processed 500/20671 rows...
  • processed 1000/20671 rows...
  • processed 1500/20671 rows...
  • processed 2000/20671 rows...
  • processed 2500/20671 rows...
  • processed 3000/20671 rows...
  • processed 3500/20671 rows...
  • processed 4000/20671 rows...
💾 Saved partial preview (4000 rows) to preview_replace_title_comp_lyrics.csv

⚙️ Processing batch 2/6 - rows 4000:8000 ...
  • processed 4500/20671 rows...
  • processed 5000/20671 rows...
  • processed 5500/20671 rows...
  • processed 6000/20671 rows...
  • processed 6500/20671 rows...
  • processed 7000/20671 rows...
  • processed 7500/20671 rows...
  • processed 8000/20671 rows...
💾 Saved partial preview (8000 rows) to preview_replace_title_comp_lyrics.csv

⚙️ Processing batch 3/6 - rows 8000:12000 ...
  • processed 8500/20671 rows...
  • processed 9000/20671 rows...
  • processed 9500/20671 rows...
  • processed 10000/20671 rows...
  • processed 1

In [6]:
import pandas as pd
import ast

def simple_fill_urls_by_index(final_file, preview_file, output_file):
    # Đọc file
    df_final = pd.read_csv(final_file)
    df_preview = pd.read_csv(preview_file)

    # Đảm bảo cột index là int
    df_preview['index'] = df_preview['index'].astype(int)

    # Dùng set cho tra cứu nhanh
    preview_map = dict(zip(df_preview['index'], df_preview['url_refer']))

    filled = 0
    for i, row in df_final.iterrows():
        if i in preview_map:
            new_url = preview_map[i]
            if isinstance(new_url, str) and new_url.strip():
                try:
                    urls = ast.literal_eval(row['urls']) if isinstance(row['urls'], str) else []
                    if not isinstance(urls, list):
                        urls = [urls]
                except Exception:
                    urls = []

                urls.append(new_url)
                # loại trùng + stringify
                df_final.at[i, 'urls'] = str(list(set(urls)))
                filled += 1

    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"✅ Đã fill lại {filled} dòng dựa theo index")
    print(f"💾 Đã lưu file kết quả: {output_file}")
    return df_final


# 🧩 Chạy lệnh
filled_df = simple_fill_urls_by_index(
    final_file="vietnamese-stopwords/final_dataset_no_nhacvn.csv",
    preview_file="preview_replace_title_comp_lyrics.csv",
    output_file="vietnamese-stopwords/final_dataset_cleaned_v3.csv"
)

✅ Đã fill lại 20671 dòng dựa theo index
💾 Đã lưu file kết quả: final_dataset_cleaned_v3.csv


In [7]:
import pandas as pd
import ast # Dùng để chuyển string "['a', 'b']" thành list ['a', 'b']
from urllib.parse import urlparse # Dùng để phân tích URL
import sys # Dùng để thoát chương trình nếu có lỗi

# --- ⬇️ BƯỚC 1: CẤU HÌNH FILE VÀ CỘT ⬇️ ---

# 1. Điền đường dẫn (path) đến file của bạn
# Ví dụ: 'data/dataset_cua_toi.csv' hoặc 'D:/Project/data.xlsx'
file_path = 'vietnamese-stopwords/final_dataset_cleaned_v3.csv'  # <-- THAY ĐỔI PATH NÀY

# 2. Điền tên chính xác của cột chứa URLs
url_column_name = 'urls' # <-- THAY ĐỔI TÊN CỘT NÀY (nếu cần)

# -----------------------------------------------

# --- BƯỚC 2: ĐỌC FILE DỮ LIỆU ---
try:
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path)
    else:
        print(f"LỖI: Không nhận diện được đuôi file. Chỉ hỗ trợ .csv, .xlsx, .xls")
        print("Đang thử đọc bằng .csv...")
        df = pd.read_csv(file_path) # Thử mặc định là CSV

    print(f"✅ Đã tải file thành công: {file_path}")

    # Kiểm tra xem cột URL có tồn tại không
    if url_column_name not in df.columns:
        print(f"❌ LỖI: Không tìm thấy cột tên là '{url_column_name}' trong file.")
        print(f"Các cột có sẵn là: {list(df.columns)}")
        sys.exit() # Dừng chương trình

except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file tại đường dẫn: {file_path}")
    sys.exit() # Dừng chương trình
except Exception as e:
    print(f"❌ LỖI: Có vấn đề khi đọc file. Lỗi: {e}")
    sys.exit() # Dừng chương trình


# --- BƯỚC 3: XỬ LÝ VÀ TRÍCH XUẤT DOMAIN (Giống như trước) ---

# Hàm chuyển string "['a']" thành list ['a'] một cách an toàn
def safe_parse_list(data_str):
    if pd.isna(data_str):
        return [] # Trả về list rỗng nếu là NaN
    try:
        # ast.literal_eval sẽ đánh giá string thành đối tượng Python
        return ast.literal_eval(data_str)
    except (ValueError, SyntaxError):
        return [] # Trả về list rỗng nếu string không đúng định dạng

# Áp dụng hàm này cho cột 'urls' của bạn
df['url_list'] = df[url_column_name].apply(safe_parse_list)

# Tách mỗi URL trong list ra thành một dòng riêng
df_exploded = df.explode('url_list')

# Hàm trích xuất domain từ mỗi URL
def get_domain(url):
    if pd.isna(url):
        return None
    try:
        # urlparse(url).netloc sẽ trả về phần domain
        # Ví dụ: 'https://nhac.vn/bai-hat/...' -> 'nhac.vn'
        return urlparse(url).netloc
    except:
        return None # Nếu URL bị lỗi

# Áp dụng hàm get_domain cho cột 'url_list'
df_exploded['domain'] = df_exploded['url_list'].apply(get_domain)

# Lấy danh sách các domain duy nhất
unique_domains = df_exploded['domain'].dropna().unique()

print("\n--- KẾT QUẢ ---")
print("Danh sách các trang web (domain) duy nhất đã cào là:")
# In ra dạng list cho dễ đọc và copy
print(list(unique_domains))

✅ Đã tải file thành công: final_dataset_cleaned_v3.csv

--- KẾT QUẢ ---
Danh sách các trang web (domain) duy nhất đã cào là:
['lyric.tkaraoke.com', 'loibaihat.co', 'nhac.vn', 'timbaihat.com', 'hopamviet.vn', 'nhachayvn.net', 'keeng.vn']
